# Imports and file reads

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import gc
import pickle
import os


from IPython.display import display
from IPython.display import Image

%load_ext autoreload
%autoreload 2

from talking_utils import *
from sklearn.cross_validation import StratifiedKFold
    
from data_preprocess import *
pbar = ProgressBar()
pbar.register()

%matplotlib inline
%config IPCompleter.greedy=True
%autosave 30

mpl.rcParams['font.size']=12                #10 
mpl.rcParams['savefig.dpi']=100             #72 
mpl.rcParams['figure.subplot.bottom']=.1 

datafiles = {}

for folder, subs, files in os.walk('af/data'):
  for filename in files:
        datafiles[filename] = os.path.abspath(os.path.join(folder, filename))
        
print(datafiles)

def save_df_pd(df,path):
    print('Args passed',path)
    df.to_csv(path,index=False)
    
def save_dictionary(dic,path):
    print('Saving to ',path)
    np.save(path, dic)
    
def save_keras_model(model,path):
    model.save(path)
    
def save_in_thread(args,func):
    save_thread = threading.Thread(target=func, args=args)
    save_thread.start()
    return save_thread

def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

make_keras_picklable()

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the 

Autosaving every 30 seconds
{'test.csv': '/home/users/nikhil/af/data/test.csv', 'test_supplement.csv.zip': '/home/users/nikhil/af/data/test_supplement.csv.zip', 'sample_submission.csv': '/home/users/nikhil/af/data/sample_submission.csv', 'train.csv': '/home/users/nikhil/af/data/mnt/ssd/kaggle-talkingdata2/competition_files/train.csv', 'test_supplement.csv': '/home/users/nikhil/af/data/data/test_supplement.csv'}


# Basic Feature Engineering

In [2]:
import pickle

#Data was preprocessed earlier with features earlier, code is in talking_utils.py
#Saved the 

with open('obj/train_np_20gp.pkl', 'rb') as handle:
    train_np = pickle.load(handle)
    
with open('obj/test_np_20gp.pkl', 'rb') as handle:
    test_np = pickle.load(handle)

with open('obj/y_np_20gp.pkl', 'rb') as handle:
    y = pickle.load(handle)

In [ ]:
# use scikit stratified test/train split for a unbiased test set
train_data,val_data,train_labels,val_labels = get_evenly_split_train_val(0.1, y.ravel(), train_np)

In [ ]:
model = get_keras_layer_model(train_np, test_np, 100, 32, 0.5,[64],[2],[4])

lent = 180903890

model = get_keras_model_compiled(lent, 2**18, 1, 0.001, 0.0001, model)

gc.collect()

In [ ]:
#Class weights due to difference in examples of both classes, ran a loop to come up with a better threshold values which reduces the classes incorrectly marked 0
class_weight = {0:1-0.6860999999999795,1:.6860999999999795}
del train_np
gc.collect()

In [ ]:
gc.collect()
model.fit(train_data, train_labels, batch_size=int((2**18)), epochs=1,shuffle=True,class_weight=class_weight,verbose=1,validation_data=(val_data, val_labels))

#saving wts when ran training, over night
model_json = model.to_json()
with open("model_WT6.json", "w") as json_file:
    json_file.write(model_json)


model.save_weights("model_wt6.h5")
save_obj(model,'model_conv_wt6')

In [ ]:
pred = model.predict(test_np,batch_size=2**19,verbose=1)

In [ ]:
df = pd.DataFrame()

In [ ]:
df['click_id'] = np.arange(0,18790469,1)

In [ ]:
df['is_attributed'] = pred

In [ ]:
df.to_csv('subm_conv_wtm.csv',index=False)